# Functions to Create Prediction Score Sheet

This jupyter notebook was created for the purpose of querying the REST API programmatically and obtain results of the predction since there were a lot of Quarter's to be analyzed.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [20]:
linearregoutfile=r"C:\Users\Manasi\Desktop\linearregoutfile.csv"
neuralnetworksoutfile=r"C:\Users\Manasi\Desktop\neuralnetwokoutfile.csv"
boosteddecisiontreeoutfiler=r"C:\Users\Manasi\Desktop\boosteddecisiontree.csv"

In [21]:
df=pd.read_csv(r"C:\Users\Manasi\Desktop\predictionTestCases.csv")

In [22]:
df.head()

,OG_QUARTER_YEAR,CREDITSCORE,FIRST_HOME_BUYER_FLAG,OG_UPB,OG_LOANTERM,SELLER_NAME,SERVICE_NAME,ORIGINAL_INTERESTRATE
0,Q1 1999,615,Y,48000,360,Other sellers,Other servicers,5.375
1,Q1 1999,791,N,284000,312,Other sellers,Other servicers,5.750
2,Q2 1999,574,N,162000,325,Other sellers,Other servicers,6.875
3,Q2 1999,716,Y,70000,360,"NORWEST MORTGAGE, IN",WELLSFARGOHOMEMORTGA,7.125
4,Q3 1999,642,N,178000,329,Other sellers,Other servicers,7.125


In [12]:
import os
import json
import requests
from flask import Flask, render_template, request, jsonify

In [35]:
def get_prediction(og_quarter_year,credit_score,og_first_time_home_buyer,og_upb,og_loan_term,og_seller_name,og_servicer_name,algoType):

        #print(str(algoType)+"\t"+str(credit_score)+"\t"+str(og_first_time_home_buyer)+"\t"+str(og_upb)+"\t"+str(og_loan_term)+"\t"+str(og_quarter_year)+"\t"+str(og_seller_name)+"\t"+str(og_servicer_name))
        #make ai call
        if algoType=="pred_df":
            url="https://ussouthcentral.services.azureml.net/workspaces/43247f6706b64e68a8b7d22ff16a2a4f/services/89403771d86f46e887c4d27b6eb80908/execute?api-version=2.0&details=true"
            api_key='4/1AxmSvNJyjoFfc1SRtNHf8Znsr7/51JGIcH4Pxb7tI+1NEWrXlXzwUwy5/k5rBDz9kLENVEpt/iY2E/Tkt3g=='
        elif algoType=="pred_nn":
            url="https://ussouthcentral.services.azureml.net/workspaces/43247f6706b64e68a8b7d22ff16a2a4f/services/964b03c9c6fa480ca1156bd093eb9ff4/execute?api-version=2.0&details=true"
            api_key="A2fylieTrkGctlekKOCkblCgcEZkxq06AHi8fuE69A9MFfVeKHfVj2uMftzuN+utNqV3Wfuyhj1HmM8F4Fj/8g=="
        elif algoType=="pred_lr":
            url="https://ussouthcentral.services.azureml.net/workspaces/43247f6706b64e68a8b7d22ff16a2a4f/services/1f553c369b314b9dad2097ff5c012200/execute?api-version=2.0&details=true"
            api_key="cz3t27xidE9ehvF8jtVhP0+s97DKnKqOLaodqaXC8I3Vmn844xIqjoAN8Q/enkiA1HXaOIvSLZQgi/SjaXfX2Q=="
        
        
        data =  {
        
                "Inputs": {
        
                        "input1":
                        {
                            "ColumnNames": ["CREDIT_SCORE", "FIRST_HOME_BUYER_FLAG", "OG_UPB", "OG_LOANTERM", "SELLER_NAME", "SERVICE_NAME", "OG_QUARTERYEAR"],
                            "Values": [ [credit_score,og_first_time_home_buyer,og_upb,og_loan_term,og_seller_name,og_servicer_name,og_quarter_year]]
                        },        },
                    "GlobalParameters": {
        }
            }
        
        body = str.encode(json.dumps(data))
        
        #url = 'https://ussouthcentral.services.azureml.net/workspaces/5de0e8bd28f74cf9a40babb3f1799a53/services/300d6267d2f843c9a5975621ff077a09/execute?api-version=2.0&details=true'
        #api_key = 'wQWgTpa3GyVACzg7Q6jVDdwt5JEDnfdvqqG21PKDr+UHmZWRQJh1XfrtLVON846vEDEXoDgnruZ1s9zd4Drzyw==' # Replace this with the API key for the web service
        headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
        
        response = requests.post(url, data=body,headers=headers)
        #print(response.content)
        
        response_json=json.loads(response.content)
        predicted_interest_rate=response_json['Results']['output1']['value']['Values'][0][7]
        
        return_dict = {"predicted_interest_rate":predicted_interest_rate}
        return predicted_interest_rate
  

In [36]:

def writePredOutputToFile(outfile,algo_type):
    with open(outfile,"w") as fil:
        for index, row in df.iterrows():
                fil.write(get_prediction(row.OG_QUARTER_YEAR,row.CREDITSCORE,row.FIRST_HOME_BUYER_FLAG,row.OG_UPB,row.OG_LOANTERM,row.SELLER_NAME,row.SERVICE_NAME,algo_type))
                fil.write("\n")

In [38]:
writePredOutputToFile(linearregoutfile,'pred_lr')
writePredOutputToFile(neuralnetworksoutfile,'pred_nn')
writePredOutputToFile(boosteddecisiontreeoutfiler,'pred_df')